In [26]:
import json
import logging
import math
import os
import pickle
import re
import torch
import warnings

import numpy as np
import pandas as pd
import skimage.io as io
import torch.nn.functional as F

from collections import defaultdict, Counter
from itertools import combinations, permutations
from numba import njit, jit, prange
from os.path import join as pjoin
from skimage.transform import resize
from torch.optim import Adam, AdamW
from torch.utils.data.distributed import DistributedSampler
from torch.utils.data import Dataset, DataLoader, SequentialSampler
from typing import Tuple, Iterator, List, Dict

In [2]:
os.environ['PYTHONIOENCODING']='UTF-8'

In [23]:
cd

/Users/ivaschenk


In [24]:
# np.load("Desktop/Thesis/master_thesis/SPoSE/data/train_90.npy")

array([[1244, 1049,  493],
       [ 887, 1787, 1249],
       [1255,  945,  733],
       ...,
       [1168,  578,  387],
       [ 123,  466,   35],
       [1765,  130,  372]])

In [29]:
def load_data(device:torch.device, triplets_dir:str, inference:bool=False) -> Tuple[torch.Tensor]:
    """load train and test triplet datasets into memory"""
    if inference:
        with open(pjoin(triplets_dir, 'test_triplets.npy'), 'rb') as test_triplets:
            test_triplets = torch.from_numpy(np.load(test_triplets)).to(device).type(torch.LongTensor)
            return test_triplets
    try:
        with open(pjoin(triplets_dir, 'train_90.npy'), 'rb') as train_file:
            train_triplets = torch.from_numpy(np.load(train_file)).to(device).type(torch.LongTensor)

        with open(pjoin(triplets_dir, 'test_10.npy'), 'rb') as test_file:
            test_triplets = torch.from_numpy(np.load(test_file)).to(device).type(torch.LongTensor)
    except FileNotFoundError:
        print('\n...Could not find any .npy files for current modality.')
        print('...Now searching for .txt files.\n')
        train_triplets = torch.from_numpy(np.loadtxt(pjoin(triplets_dir, 'train_90.txt'))).to(device).type(torch.LongTensor)
        test_triplets = torch.from_numpy(np.loadtxt(pjoin(triplets_dir, 'test_10.txt'))).to(device).type(torch.LongTensor)
    return train_triplets, test_triplets

#load triplets into memory
train_triplets, test_triplets = load_data(device='cuda', triplets_dir="Desktop/Thesis/master_thesis/SPoSE/data")

AssertionError: Torch not compiled with CUDA enabled

In [33]:
torch.cuda.is_available()

False